In [1]:
R = QQ['q'].fraction_field()
q = R.gen()
NCSym = SymmetricFunctionsNonCommutingVariables(R)
chi = NCSym.chi(q)

In [2]:
def crs(x, y):
    crs = 0
    for i in range(len(x)):
        for j in range(len(y)):
            if x[i][0] < y[j][0] < x[i][1] < y[j][1]:
                crs = crs+1    
    return crs

In [3]:
def nesting(x, y):
    if y[0] < x[0] < x[1] < y[1]:
        return True

In [4]:
def res_coeff(x, y):
    return (q-1)^(Set(x).difference(Set(y)).cardinality())*q^(crs(Set(x).intersection(Set(y)),Set(x).difference(Set(y))))/q^(crs(Set(y).intersection(Set(x)),Set(y).difference(Set(x))))

In [5]:
def ind_coeff(x, y):
    return (q-1)^(Set(x).difference(Set(y)).cardinality())*q^(crs(Set(x).difference(Set(y)),Set(x).intersection(Set(y))))/q^(crs(Set(y).difference(Set(x)),Set(y).intersection(Set(x))))

In [6]:
def getKey(item):
    return item[1]

In [7]:
def Res(n,self):
# self = supercharacter given by [[partition], coeff]
    arcsort = sorted(self[0], key=getKey)
    if arcsort == []:
        return [self]
    if arcsort[-1][1] != n:
        return [self]
    elif arcsort[-1][1] == n:
        spiral = arcsort.pop(-1)
        l = []
        l.append(spiral)
        for _ in self[0]:
            if _[0] < spiral[0]:
                l.append(_)
        P = Poset((self[0], nesting), cover_relations=False)
        C = list(P.chains(exclude=[_ for _ in l]))
        S = [_+[spiral] for _ in C]
        res = []
        re = [max(_) for _ in self[0]]
        for i in range(len(S)):
            arcs = self[0][:]
            for j in range(len(S[i])):
                arcs.remove(S[i][j])
            for j in range(len(S[i])-1):
                arcs.append((S[i][j+1][0], S[i][j][1]))
            res.append(arcs)
            for k in range(S[i][0][1]-S[i][0][0]-1):
                if S[i][0][1]-k-1 not in re:
                    res.append(arcs+[(S[i][0][0],S[i][0][1]-k-1)])         
        return [[res[i], res_coeff(self[0], res[i])*self[1]] for i in range(len(res))]

In [8]:
def Ind(n, self):
# self = supercharacter given by [[partition], coeff]
    P = Poset((self[0], nesting), cover_relations=False)
    C = list(P.chains())
    ind = [self[0]]
    le = [min(_) for _ in self[0]]
    for j in range(1,n):    
        if j not in le:
            ind.append(self[0]+[(j,n)])
    for j in range(1,len(C)):
        arcs = self[0][:]
        for k in range(len(C[j])):
            arcs.remove(C[j][k])
        for k in range(len(C[j])-1):
            arcs.append((C[j][k][0], C[j][k+1][1]))
        arcs.append((C[j][len(C[j])-1][0],n))
        ind.append(arcs)
        for k in range(C[j][0][1]-C[j][0][0]-1):
            if C[j][0][0]+k+1 not in le:
                ind.append(arcs+[(C[j][0][0]+k+1,C[j][0][1])])       
    return [[ind[i],ind_coeff(self[0], ind[i])*self[1]] for i in range(len(ind))]

In [9]:
def ResSum(n, self):
    l = map(Res, [n]*len(self), self)
    d = [[tuple(sorted(item[0], key=getKey)), item[1]] for sublist in l for item in sublist]
    simplify = {}
    for k, v in d:
        simplify[k] = simplify.get(k, 0) + v
    return [[list(item[0]), item[1]] for item in list(simplify.items())]

In [10]:
def IndSum(n, self):
    l = map(Ind, [n]*len(self), self)
    d = [[tuple(sorted(item[0], key=getKey)), item[1]] for sublist in l for item in sublist]
    simplify = {}
    for k, v in d:
        simplify[k] = simplify.get(k, 0) + v
    return [[list(item[0]), item[1]] for item in list(simplify.items())]

In [11]:
def IndRes(n,k):
    def funk(y):
        return reduce(lambda x, _: IndSum(n,ResSum(n,x)), xrange(k), y)
    return funk

In [12]:
IndRes(4,3)([[[],1]])

[[[(2, 3), (3, 4)], 2*q^3 - q^2 - 2*q + 1],
 [[(1, 2)], 2*q^3 - q^2 - 2*q + 1],
 [[(2, 3)], 2*q^3 - q^2 - 2*q + 1],
 [[(1, 2), (2, 3)], q^3 - q^2 - q + 1],
 [[(2, 4)], 2*q^4 - q^2],
 [[(3, 4)], 3*q^3 - 3*q + 1],
 [[(1, 3)], q^4 - q^2],
 [[(1, 4)], q^5 + q^4 - q^3],
 [[(1, 2), (2, 3), (3, 4)], q^3 - q^2 - q + 1],
 [[(1, 3), (3, 4)], q^4 - q^2],
 [[(1, 3), (2, 4)], q^4 - q^2],
 [[], 3*q^3 - 3*q + 1],
 [[(1, 2), (2, 4)], q^4 - q^2],
 [[(2, 3), (1, 4)], q^5 - q^3],
 [[(1, 2), (3, 4)], 2*q^3 - q^2 - 2*q + 1]]